In [1]:
import re

import numpy as np

from IPython.display import display, Markdown
from pathlib import Path
from collections import defaultdict
from itertools import chain

In [2]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

In [3]:
BIODIVERSITY_PATTERN = re.compile(r"The simulation resulted in a biodiversity of (\d+) unique species\.")
EXECUTION_PATTERN = re.compile(r"The simulation took:\n - initialisation: ([^\n]+)\n - execution: ([^\n]+)\n - cleanup: ([^\n]+)\n")
EVENT_PATTERN = re.compile(r"Event Summary:\n - Total #individuals:\n   \d+\n - Total #events:\n   - raw:\n     (\d+)")

In [4]:
TIME_PATTERN = re.compile(r"(\d+(?:\.\d+)?)([^\d]+)")
TIME_UNITS = {
    "ns": 0.000000001,
    "µs": 0.000001,
    "ms": 0.001,
    "s": 1.0,
}

def parse_time(time_str):
    match = TIME_PATTERN.match(time_str)
    
    if match is None:
        return None
    
    return float(match.group(1)) * TIME_UNITS[match.group(2)]

In [5]:
DOMAIN_DIR_PATTERN = re.compile(r"([+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?)")
DOMAIN_FILE_PATTERN = re.compile(r"([+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?)/limit\.isolated\.o\d+\.\d+")
DOMAIN_REPLAY_PATTERN = re.compile(r"([+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?)/limit\.\d+(?:\[\])?\.pbs\.o\d+")

In [6]:
display(Markdown("# Scalability of the IsolatedLandscape Independent Algorithm variant:"))

throughput_results = defaultdict(list)
redundancy_results = defaultdict(list)

for path in Path(".").iterdir():
    match = DOMAIN_DIR_PATTERN.match(str(path))
    
    if match is None:
        continue
    
    for path in Path(path).iterdir():
        match = DOMAIN_FILE_PATTERN.match(str(path))
        
        if match is not None:
            sample = float(match.group(1))

            with open(path) as file:
                stdout = file.read()

            match = EXECUTION_PATTERN.search(stdout)
            if match is None:
                print(path)
                continue
            initialisation = parse_time(match.group(1))
            execution = parse_time(match.group(2))
            cleanup = parse_time(match.group(3))

            throughput_results[sample].append((initialisation, execution, cleanup))
        
        match = DOMAIN_REPLAY_PATTERN.match(str(path))
        
        if match is not None:
            sample = float(match.group(1))
            
            with open(path) as file:
                stdout = file.read()
                
            match = BIODIVERSITY_PATTERN.search(stdout)
            if match is None:
                print(path)
                continue
            biodiversity = int(match.group(1))
            
            match = EVENT_PATTERN.search(stdout)
            if match is None:
                print(path)
                continue
            raw_speciations = int(match.group(1))
            
            redundancy_results[sample].append((biodiversity, raw_speciations, raw_speciations / biodiversity))

results = { domain: [
    (np.median(s), np.mean(s), np.std(s), len(s)) for s in chain(zip(*result), zip(*redundancy_results[domain]))
] for domain, result in throughput_results.items() }

(
    domains, initialisations, executions, cleanups, biodiversities, raw_speciations, redundancies
) = zip(*sorted([domain] + rs for domain, rs in results.items()))

for domain, execution, redundancy in zip(domains, executions, redundancies):
    display(Markdown(fr"* ${domain}$ individuals: ${np.round(execution[1], 2)}s \pm {np.round(execution[2], 2)}s$ per batch [{execution[3]} samples] -> ${np.round(redundancy[1] * 100, 2)}\% \pm {np.round(redundancy[2] * 100, 2)}\%$"))

# Scalability of the IsolatedLandscape Independent Algorithm variant:

* $1000000000.0$ individuals: $13722.85s \pm 2540.23s$ per batch [100 samples] -> $164.19\% \pm 0.25\%$

* $10000000000.0$ individuals: $5584.37s \pm 2276.5s$ per batch [1000 samples] -> $453.93\% \pm 0.54\%$

* $71000000000.0$ individuals: $3617.18s \pm 2138.75s$ per batch [7100 samples] -> $1602.39\% \pm 2.57\%$